In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from scipy.optimize import differential_evolution

warnings.filterwarnings("ignore")

# Load data
df = pd.read_excel("/content/drive/MyDrive/Mansoor_Sir_Project/Merged_Dataset.xlsx",
                  sheet_name="Sheet1", engine="openpyxl")

# Define input and output
input_x = df.iloc[:, [0, 1, 2]].astype('float32')
output_y = df.iloc[:, [3]].astype('float32')

# Split and scale
test_size = 0.2
x_train, x_test, y_train, y_test = train_test_split(input_x.values, output_y.values, test_size=test_size, random_state=42)
scaler_x = StandardScaler()
scaler_y = StandardScaler()

x_train_scaled = scaler_x.fit_transform(x_train)
y_train_scaled = scaler_y.fit_transform(y_train)
x_test_scaled = scaler_x.transform(x_test)
y_test_scaled = scaler_y.transform(y_test)

# Model
def build_model():
    model = Sequential([
        Dense(64, input_shape=(x_train_scaled.shape[1],)),
        LeakyReLU(alpha=0.1),
        Dense(32),
        LeakyReLU(alpha=0.1),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mse',
                  metrics=['mae', 'mse'])
    return model

model = build_model()

history = model.fit(
    x_train_scaled, y_train_scaled,
    epochs=1000,
    batch_size=36,
    validation_split=0.2,
    verbose=1
)

# Predictions
y_pred_scaled = model.predict(x_test_scaled)
y_pred_denorm = scaler_y.inverse_transform(y_pred_scaled)
y_test_denorm = scaler_y.inverse_transform(y_test_scaled)

# Evaluation
mae_score = mean_absolute_error(y_test_denorm, y_pred_denorm)
mse_score = mean_squared_error(y_test_denorm, y_pred_denorm)
r2 = r2_score(y_test_denorm, y_pred_denorm)
print(f'R2 Score: {r2:.4f}')
print(f'MAE: {mae_score:.4f}')
print(f'MSE: {mse_score:.4f}')

# Plots
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss (MSE)')
plt.plot(history.history['val_loss'], label='Validation Loss (MSE)')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.title('Training and Validation MSE Loss')
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Error (MAE)')
plt.title('Training and Validation MAE')
plt.legend()
plt.grid()
plt.show()

# Actual vs Predicted in mW
y_test_mW = 10 ** (y_test_denorm.flatten() / 10)
y_pred_mW = 10 ** (y_pred_denorm.flatten() / 10)
plt.figure(figsize=(10, 6))
plt.scatter(y_test_mW, y_pred_mW, alpha=0.6)
plt.plot([y_test_mW.min(), y_test_mW.max()], [y_test_mW.min(), y_test_mW.max()], 'r--', label="Perfect Fit")
plt.xlabel('Actual Gain (mW)')
plt.ylabel('Predicted Gain (mW)')
plt.title('Actual vs Predicted Gain in mW')
plt.legend()
plt.grid()
plt.show()

# Actual vs Predicted in dB
plt.figure(figsize=(10, 6))
plt.scatter(y_test_denorm, y_pred_denorm, alpha=0.6)
plt.plot([y_test_denorm.min(), y_test_denorm.max()], [y_test_denorm.min(), y_test_denorm.max()], 'r--', label="Perfect Fit")
plt.xlabel('Actual Gain (dB)')
plt.ylabel('Predicted Gain (dB)')
plt.title('Actual vs Predicted Gain in dB')
plt.legend()
plt.grid()
plt.show()

# Optimization

def find_pump_config(target_gain, central_signal_wavelength):
    num_neighbors = 50
    wavelength_range = np.linspace(central_signal_wavelength - 25, central_signal_wavelength + 25, num_neighbors)

    def loss_function(params):
        pump_power, pump_wavelength = params
        input_data = np.array([[pump_wavelength, pump_power, w] for w in wavelength_range])
        input_data_scaled = scaler_x.transform(input_data)
        predicted_gains = model.predict(input_data_scaled, verbose=0)
        predicted_gains = scaler_y.inverse_transform(predicted_gains).flatten()
        mse_loss = np.mean((predicted_gains - target_gain) ** 2)
        variance_penalty = np.var(predicted_gains)
        negative_penalty = np.sum(np.abs(np.minimum(predicted_gains, 0))) * 10
        return mse_loss + 0.1 * variance_penalty + negative_penalty

    bounds = [(100, 800), (1300, 1520)]
    result = differential_evolution(loss_function, bounds, strategy='best1bin', maxiter=200, popsize=15, tol=0.01,
                                    mutation=(0.5, 1.9), recombination=0.8)
    return result.x if result.success else None

# Predict loop
target_gain = 25
signal_wavelengths = np.arange(1370, 1406, 1)
results = []

for signal_wavelength in signal_wavelengths:
    solution = find_pump_config(target_gain, signal_wavelength)
    if solution is None:
        results.append([signal_wavelength, None, None, None])
        continue
    pump_power, pump_wavelength = solution
    if not (100 <= pump_power <= 800) or not (1300 <= pump_wavelength <= 1520):
        results.append([signal_wavelength, None, None, None])
        continue
    input_data = np.array([[pump_wavelength, pump_power, signal_wavelength]])
    input_data_scaled = scaler_x.transform(input_data)
    predicted_gain = scaler_y.inverse_transform(model.predict(input_data_scaled, verbose=0))
    results.append([signal_wavelength, pump_wavelength, pump_power, predicted_gain[0][0]])

results_df = pd.DataFrame(results, columns=['Signal Wavelength', 'Predicted Pump Wavelength', 'Predicted Pump Power', 'Predicted Gain'])
valid_results = results_df.dropna()
print("\nValid Predictions:")
print(valid_results)

plt.figure(figsize=(12, 6))
plt.plot(valid_results['Signal Wavelength'], valid_results['Predicted Gain'], 'o-')
plt.axhline(y=target_gain, color='r', linestyle='--', label='Target Gain')
plt.xlabel('Signal Wavelength (nm)')
plt.ylabel('Predicted Gain (dB)')
plt.title(f'Predicted Gain vs Signal Wavelength (Target: {target_gain} dB)')
plt.legend()
plt.grid(True)
plt.show()